<a href="https://colab.research.google.com/github/NattachaiWat/ml442/blob/master/Worksheet3_VGG16_Caltech101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Workshop 3.3: Lab experiments

## Objectives:

    1. Learning about Tensorflow Dataset (Caltech101)
    (https://www.tensorflow.org/datasets/catalog/caltech101)
    2. Create a VGG using Sequential Model
    3. Try to train


In [0]:
#@title Repeat! import tensorflow 2.x again
!pip install -q tf-nightly-2.0-preview
%load_ext tensorboard

from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# Load the TensorBoard notebook extension

tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [0]:
#@title We have to import tfds
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
#@title Try to visit caltech101 web  { vertical-output: true }
#@markdown http://www.vision.caltech.edu/Image_Datasets/Caltech101/
split_train = tfds.Split.TRAIN.subsplit(tfds.percent[:80])
split_test  = tfds.Split.TRAIN.subsplit(tfds.percent[-20:])

#@markdown Enter dataset name from tfds
dataset_name = '' #@param {type:'string'}
tfds_train = tfds.load(name=dataset_name, 
                             split=split_train,
                             data_dir='./dataset')
tfds_valid   = tfds.load(name=dataset_name,
                               split=split_test,
                               data_dir='./dataset')
tfds_builder = tfds.builder(dataset_name)
tfds_info  = tfds_builder.info
label_names = tfds_info.features['label'].names
nClass = tfds_info.features['label'].num_classes

In [0]:
print(type(caltech101_train))

In [0]:
#@title Explore images and label in dataset { vertical-output: true }
nrows = 3
ncols = 3
plt.figure(figsize=(ncols*3,nrows*3))
for i,data in enumerate (tfds_train.take(nrows*ncols)):  # Only take a single example
  image, label = data["image"], data["label"]
  plt.subplot(nrows,ncols,i+1)
  plt.imshow(image)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.title('{}:{}'.format(label_names[label],image.shape))





In [0]:
#@title Normalizing and Resizeing image from tf.dataset { vertical-output: true }
#@markdown Enter  input size of image
resize_img = None #@param 
#@markdown Hint: 2 dimensions

def image_process(data):
  img = tf.image.resize(data['image'],size=resize_img)
  img = tf.divide(img,255.0)
  label = tf.one_hot(data['label'],nClass)
  return img,label

ds_train = tfds_train.map(image_process)
ds_val = tfds_valid.map(image_process)

nrows = 3
ncols = 3
plt.figure(figsize=(ncols*3,nrows*3))
for i,data in enumerate (ds_train.take(nrows*ncols)):  # Only take a single example
  image, label = data[0], data[1]
  label = int(tf.argmax(label).numpy())
  plt.subplot(nrows,ncols,i+1)
  plt.imshow(image)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.title('{}:{}'.format(label_names[label],image.shape))


## VGG16 as Sequential API

Next, we will create a VGG convolutional network. VGG networks are sequential, but they add the concept of convolutional groups. The basic elements of a VGG are:

    1. Each convolutional group consists of two or more convolutional layers.
    2. Max pooling is deferred to the end of the convolutional group.
    3. Each convolutional group is the same or double the number of filters as the last  
       group.
    4. Multiple dense layers are used for the classifer.

In [0]:
#@title Construct VGG16 { vertical-output: true }

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def conv_block(n_layers, n_filters):
    """
        n_layers : number of convolutional layers
        n_filters: number of filters
    """
    for n in range(n_layers):
        model.add(Conv2D(n_filters, (3, 3), strides=(1, 1), 
                  padding="same",
                  activation="relu"))
    model.add(MaxPooling2D(2, strides=2))

# Create a Sequential Model
model = Sequential()

# Add Convolutional Frontend with 64 3x3 filters of stride 1
# Set the padding so when the filter is slid over the edges of the image, the "imaginary" pixels have the same
# value as the pixels on the edge.

#@markdown Enter shape of input image
input_shape= None #@param 
model.add(Conv2D(filters=64, 
          kernel_size=(3,3), strides=(1, 1), 
          padding='same', activation="relu",
          input_shape=input_shape))


#@markdown  These are the convolutional groups - double the number of filters on each progressive group

#@markdown Enter the number of filter in the first block
nfilter_in_block1 = None #@param 
conv_block(1, nfilter_in_block1)
conv_block(2, nfilter_in_block1*2)
conv_block(3, nfilter_in_block1*4)

#@markdown The last two groups in a VGG16, its double the size of the previous of the group, but both groups are the same size.

#@markdown Enter the number of filter in the first block
nfilter_in_block2 = None  #@param 
conv_block(3, nfilter_in_block2*8)
conv_block(3, nfilter_in_block2*8)

#@markdown  Add DNN Backend with two layers of 4096 nodes
# HINT: think of what you need to do to the 2D feature maps from the convolutional layers before passing to dense layers.
model.add(Flatten())

#@markdown Enter the number of node for fully connect layer
num_node1 =  None #@param 
num_node2 =  None #@param 
model.add(Dense(num_node1, activation='relu'))
model.add(Dense(num_node2, activation='relu'))

# Output layer for classification (102 classes)
#@markdown Enter the number of class for classification
num_class = None #@param 
#@markdown Enter the activation function for classification
final_activation =  None #@param 

model.add(Dense(num_class, activation=final_activation))
model.summary()


# Try to train

In [0]:
#@title Compile the model { vertical-output: true }
#@markdown Enter loss 

loss_name = '' #@param {type:'string'}
#@markdown Hint: categorical_crossentropy may not work. Why?

metric_name = '' #@param {type: 'string'}
optimizer_name =  None #@param

model.compile(loss=loss_name,  
              metrics=[metric_name], 
              optimizer=optimizer_name)

In [0]:
#@title Training the model in history { form-width: "350px" }

#@markdown Enter the batch size
batch_size = None #@param {type:'integer'}
#@markdown Enter the number of epoch
epochs = None #@param {type:'integer'}

_train = ds_train.batch(batch_size)
_val   = ds_val.batch(batch_size)

# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
_train = _train.prefetch(tf.data.experimental.AUTOTUNE)
_val   = _val.prefetch(tf.data.experimental.AUTOTUNE)


history = model.fit(_train,
          epochs=epochs,
          verbose=1,
          validation_data=_val)